In [ ]:
## pulls books from specified goodreads user shelf  

In [1]:
import pandas as pd
import numpy as np
import selenium
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [70]:
def get_star_count(rating): 
    if rating == "": 
        stars= np.nan
    if rating == "did not like it": 
        stars = 1
    if rating =="it was ok": 
        stars = 2
    if rating == "liked it": 
        stars = 3
    if rating =="really liked it": 
        stars = 4
    if rating == 'it was amazing':
        stars = 5
    return stars 

In [94]:
def clean_element(element): 

    title_full= element.find_element(By.CSS_SELECTOR, '[class="field title"]').text
    title= title_full.split("(")[0]
    author_full = element.find_element(By.CSS_SELECTOR, '[class="field author"]').text
    author_full= author_full.replace("*", "")
    author_full = author_full.split(",")
    author= author_full[1] + " " + author_full[0]

    rating_raw= element.find_element(By.CSS_SELECTOR, '[class="field rating"]').find_element(By.CSS_SELECTOR, '[class=" staticStars notranslate"]').text
    rating = get_star_count(rating_raw)
    return [title.strip(), author.strip(), rating]


In [72]:
#clean_element(review_elements[10])

In [82]:
# open driver - no need to log-in book list is public! 
driver = webdriver.Chrome()

In [95]:
book_list=[]
for i in range (1,5): 
    driver.get(f"https://www.goodreads.com/review/list/187059797-cara-n?page={i}&shelf=romantasyreads")
    try: 
        x_button= driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]")
        x_button.click()
    except: 
        pass
    review_elements = driver.find_elements(By.CSS_SELECTOR, '[id^="review_"]')

    for i in range (2, len(review_elements)): 
        title, author, rating= clean_element(review_elements[i]) 
        book_list.append([title,author,rating])
    time.sleep(5)

In [113]:
#book_list= []
for i in range (17,26): 
    driver.get("https://www.goodreads.com/shelf/show/fantasy-romance?page="+str(i))
    elements = driver.find_elements(By.CLASS_NAME, 'elementList')

    for element in elements:
        try: 
            book_list.append(clean_element(element))
        except IndexError: # last 2 are weird, not actual books w/ authors, skip 
            pass

    time.sleep(5)



In [103]:
df = pd.DataFrame(book_list, columns = ['Title', 'Author', 'Rating']) 


In [100]:
#df.to_csv("user_read_list_fantasy_romance_titles_authors.csv", index=False)

In [134]:
# read in data we have on books! 
B_df= pd.read_csv("/Users/caranix/Documents/GitHub/RomantasyRec/cleaned_romantasy_data.csv")

In [135]:
B_df

,api,Title,cleaned_title,cleaned_author,Rating,Count of Ratings,cleaned_description,cleaned_publisher,cleaned_page_count_v1,cleaned_count_ratings_v1,published_year,mature
0,https://www.googleapis.com/books/v1/volumes?q=...,A Court of Thorns and Roses,court thorn rose,sarah j maas,4.18,"3,459,145",first instalment global phenomenon tiktok sens...,bloomsbury,450+,1,2020.0,0
1,https://www.googleapis.com/books/v1/volumes?q=...,A Court of Mist and Fury,court mist fury,sarah j maas,4.65,"2,625,974",new york time bestselling sequel sarah j maass...,bloomsbury,450+,1,2017.0,0
2,https://www.googleapis.com/books/v1/volumes?q=...,Fourth Wing,fourth wing,rebecca yarros,4.58,"2,508,672",war grows deadly violet sorrengail join elite ...,entangled,<300,1,2023.0,0
3,https://www.googleapis.com/books/v1/volumes?q=...,A Court of Wings and Ruin,court wing ruin,sarah j maas,4.47,"2,194,491",feyre must decide trust among high lord turn a...,bloomsbury,450+,1,2017.0,0
4,https://www.googleapis.com/books/v1/volumes?q=...,From Blood and Ash,blood ash,jennifer l armentrout,4.22,"708,676",captivating actionpacked blood ash sexy addict...,other,300-450,1,2020.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1061,https://www.googleapis.com/books/v1/volumes?q=...,The Scorpio Races,scorpio race,maggie stiefvater,4.10,"111,812",every november scorpio race run beneath chalk ...,other,<300,2,2022.0,0
1062,https://www.googleapis.com/books/v1/volumes?q=...,Blaze of Memory,blaze memory,nalini singh,3.96,"24,997",nalini singh return psychangeling world woman ...,other,300-450,5,2011.0,0
1063,https://www.googleapis.com/books/v1/volumes?q=...,Split or Swallow,split swallow,lindsay straube,3.83,"5,662",internet sensation amazon bestseller lindsay s...,other,450+,6,2025.0,0
1064,https://www.googleapis.com/books/v1/volumes?q=...,Autumn of the Grimoire,autumn grimoire,jl vampa,4.05,"3,185",witch cast spell others slay king mysterious g...,other,300-450,6,2022.0,0


In [136]:
def clean_title (title): 
    title = title.replace("’", '')
    title= title.replace('ü', 'u')
    return title
def clean_author(author): 
    author= author.replace('.', '')
    if 'é' in author: 
        author = author.replace('é', 'e')
    if 'ü' in author: 
        author = author.replace('ü', 'u')
    if "ñ" in author: 
        author = author.replace('ñ', 'n')
    return author

In [137]:
df['cleaned_title']= df['Title'].apply(clean_title)   
df['cleaned_author']= df['Author'].apply(clean_author)   

In [138]:
new_df = pd.merge(
    left=df, 
    right=B_df,
    how='left',
    left_on=['cleaned_title'],
    right_on=['Title'],
) # note there are some which do not match! why is this? because they didn't match to google books-- so need to exclude! 


In [139]:
new_df

,Title_x,Author,Rating_x,cleaned_title_x,cleaned_author_x,api,Title_y,cleaned_title_y,cleaned_author_y,Rating_y,Count of Ratings,cleaned_description,cleaned_publisher,cleaned_page_count_v1,cleaned_count_ratings_v1,published_year,mature
0,"Hush, Hush",Becca Fitzpatrick,NaN,"Hush, Hush",Becca Fitzpatrick,https://www.googleapis.com/books/v1/volumes?q=...,"Hush, Hush",hush hush,becca fitzpatrick,3.92,"725,940",enter realm fallen angel rising passion boxed ...,other,450+,1.0,2013.0,0.0
1,Electric Idol,Katee Robert,5.0,Electric Idol,Katee Robert,https://www.googleapis.com/books/v1/volumes?q=...,Electric Idol,electric idol,katee robert,3.95,"111,306",instant new york time usa today bestseller bea...,other,<300,2.0,2022.0,1.0
2,Ruin and Rising,Leigh Bardugo,NaN,Ruin and Rising,Leigh Bardugo,https://www.googleapis.com/books/v1/volumes?q=...,Ruin and Rising,ruin rising,leigh bardugo,3.98,"559,718",grishaverse coming netflix soon shadow bone or...,other,450+,1.0,2014.0,0.0
3,The Ex Hex,Erin Sterling,5.0,The Ex Hex,Erin Sterling,https://www.googleapis.com/books/v1/volumes?q=...,The Ex Hex,ex hex,erin sterling,3.48,"248,969",new york time bestseller erin sterling cast de...,harper collins,<300,1.0,2021.0,1.0
4,A Game of Fate,Scarlett St. Clair,5.0,A Game of Fate,Scarlett St Clair,https://www.googleapis.com/books/v1/volumes?q=...,A Game of Fate,game fate,scarlett st clair,4.00,"84,809",discover enthralling fantasy world god mortal ...,other,300-450,2.0,2021.0,1.0
5,Heartless Sky,Caroline Peckham,3.0,Heartless Sky,Caroline Peckham,https://www.googleapis.com/books/v1/volumes?q=...,Heartless Sky,heartless sky,caroline peckham,4.21,"102,162",zodiac academy series twisted sister amazon wa...,other,<300,2.0,2025.0,0.0
6,The Selection,Kiera Cass,5.0,The Selection,Kiera Cass,https://www.googleapis.com/books/v1/volumes?q=...,The Selection,selection,kiera cass,4.08,"1,661,933",fall lovefrom beginning discover first book ca...,harper collins,300-450,1.0,2012.0,0.0
7,Cinder,Marissa Meyer,4.0,Cinder,Marissa Meyer,https://www.googleapis.com/books/v1/volumes?q=...,Cinder,cinder,marissa meyer,4.13,"970,130",new york time bestselling series human android...,other,300-450,1.0,2012.0,0.0
8,Ignite Me,Tahereh Mafi,5.0,Ignite Me,Tahereh Mafi,https://www.googleapis.com/books/v1/volumes?q=...,Ignite Me,ignite,tahereh mafi,4.46,"573,801",heartstopping third installment new york time ...,harper collins,<300,1.0,2014.0,0.0
9,Siege and Storm,Leigh Bardugo,5.0,Siege and Storm,Leigh Bardugo,https://www.googleapis.com/books/v1/volumes?q=...,Siege and Storm,siege storm,leigh bardugo,3.81,"649,201",see grishaverse come life screen netflix serie...,other,300-450,1.0,2013.0,0.0


In [140]:
matched_books= new_df[new_df['api'].notna()] #-- future iteration would be trying to pull this data from google api 

matched_books.to_csv("user_read_list_fantasy_romance_titles_authors.csv", index=False)

In [ ]:
# goal is now to find books from the first list, fantasy_romance_titles_authors.csv, which are similar to user_read_list_fantasy...csv 